In [1]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
from scipy.interpolate import griddata

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
weatherHist = pd.read_csv("../../data/historical_weather.csv")
weatherPred = pd.read_csv("../../data/forecast_weather.csv")

In [ ]:
weatherPred['datetime'] = pd.to_datetime(weatherPred['forecast_datetime']).dt.tz_convert(None)
weatherHist['datetime'] = pd.to_datetime(weatherHist['datetime'])
histWeatherTrain = pd.merge(weatherPred, weatherHist, on=['data_block_id','latitude','longitude','datetime'], how='inner',suffixes=('_pred', '_hist'))

In [ ]:
histWeatherTrain.drop_duplicates(inplace=True)

In [ ]:
# interpolate to county centers

interpolatedVal = []
interpolationFeat = [
       'temperature_pred', 'dewpoint_pred', 'cloudcover_high_pred',
       'cloudcover_low_pred', 'cloudcover_mid_pred', 'cloudcover_total_pred',
       '10_metre_u_wind_component', '10_metre_v_wind_component',
       'direct_solar_radiation_pred',
       'surface_solar_radiation_downwards', 'snowfall_pred',
       'total_precipitation', 'temperature_hist', 'dewpoint_hist',
       'rain', 'snowfall_hist', 'surface_pressure', 'cloudcover_total_hist',
       'cloudcover_low_hist', 'cloudcover_mid_hist', 'cloudcover_high_hist',
       'windspeed_10m', 'winddirection_10m', 'shortwave_radiation',
       'direct_solar_radiation_hist', 'diffuse_radiation'
]

# Coordinates of points where you want to interpolate
interp_points = np.array([
    [59.31092599057606, 24.924347516442776],#"HARJUMAA",        # index = county id
    [58.89325336822078, 22.61868018616204], #"HIIUMAA",         # index = county id
    [59.177732657186525, 27.327483563720584], #"IDA-VIRUMAA",   # index = county id
    [58.9401960343481, 25.66666660100041],#   "JÄRVAMAA",       # index = county id
    [58.722190565952324, 26.536903689900708],#"JÕGEVAMAA",      # index = county id
    [59.265951945347595, 26.355665774943418],#"LÄÄNE-VIRUMAA",  # index = county id
    [58.90131388796995, 23.760348418177717],#"LÄÄNEMAA",         # index = county id
    [58.39642190189904, 24.550321648922793],#"PÄRNUMAA",         # index = county id
    [58.043991137798386, 27.162798202382948],#"PÕLVAMAA",        # index = county id
    [58.923144729594874, 24.70288373555266],#"RAPLAMAA",         # index = county id
    [58.39256707482195, 22.537559658585536],#"SAAREMAA",         # index = county id
    [58.392720495713554, 26.82996683757416],#"TARTUMAA",         # index = county id
    [58.5975, 24.9873], #"UNKNOWN", = center of the country      # index = county id
    [57.93303855829815, 26.14302669183717], #"VALGAMAA",         # index = county id
    [58.336987487787404, 25.558364059539933],#"VILJANDIMAA",     # index = county id
    [57.74795525256318, 26.91115291686715], #"VÕRUMAA"           # index = county id
  ])
# Convert to a 2D array
interp_points_arr = np.vstack(interp_points)

i = 0
print('unique days', len(histWeatherTrain.forecast_datetime.unique())/10) #10 rows per day
for time_id,time in enumerate(histWeatherTrain.forecast_datetime.unique()):
  timeSlice = histWeatherTrain.loc[histWeatherTrain.forecast_datetime == time]
  points = timeSlice[['latitude', 'longitude']].values

  if (len(points)) != 110:
    print('more datapoints than we should have', time_id, time, len(points))
    break

  hours_ahead = timeSlice['hours_ahead'].unique()
  data_block_id = timeSlice['data_block_id'].unique()
  origin_datetime = timeSlice['origin_datetime'].unique()

  if len(hours_ahead) != 1 or len(data_block_id) != 1 or len(origin_datetime) != 1:
    print('more than one unique value', hours_ahead, data_block_id, origin_datetime)
  
  if i%(10*109)== 0 and i>0:
    print(i/100,' days done')
    if i%(10*1000)== 0:
      print('intermediate saving..')
      inthistWeatherTrain = pd.concat(interpolatedVal)
      inthistWeatherTrain.to_csv("interpolHistOverlapWeather.csv")
      print('done')


  allIntFeat = {}
  allIntFeat['latitude'] = interp_points_arr[:,0]
  allIntFeat['longitude'] = interp_points_arr[:,1]
  allIntFeat['County'] = np.linspace(0,15,16).astype('int')
  allIntFeat['forecast_datetime'] = time
  allIntFeat['hours_ahead']       = hours_ahead[0]
  allIntFeat['data_block_id']     = data_block_id[0]
  allIntFeat['origin_datetime']   = origin_datetime[0]

  for feat in interpolationFeat:
    # Values at the known points
    values_feat = timeSlice[feat].values

    # Linear interpolation for temperature
    result = griddata(points, values_feat, interp_points, method='linear')

    allIntFeat[feat+'int'] = result
  
  df = pd.DataFrame(allIntFeat)
  interpolatedVal.append(df)
  i = i+1

inthistWeatherTrain = pd.concat(interpolatedVal)
inthistWeatherTrain.to_csv("interpolHistOverlapWeather.csv")